In [16]:
# !pip install tensorflow==2.4.1
# !pip install transformers
# !pip install pyarrow

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from math import ceil

In [2]:
def check_targets(targs):
    if targs[0] == -1:
        return 1
    else:
        return 0

In [3]:
def create_tfrecords_dataset(data, iter_num, dataset_type='train'):
#     paper_title = tf.keras.preprocessing.sequence.pad_sequences(data['paper_title_tok'].to_list(), 
#                                                                 maxlen=512, dtype='int64', 
#                                                                 padding='post', truncating='post', value=0)
    data['no_target'] = data['target_tok'].apply(check_targets)
    data = data[data['no_target']==0].copy()
    
    paper_title = tf.ragged.constant(data['paper_title_tok'].to_list())
    
    targets = tf.keras.preprocessing.sequence.pad_sequences(data['target_tok'].to_list(), maxlen=20, 
                                                            dtype='int64', padding='post', 
                                                            truncating='post', value=0)

    ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(paper_title),
                              tf.data.Dataset.from_tensor_slices(data['journal_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(data['doc_type_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(targets)))
    
    serialized_features_dataset = ds.map(tf_serialize_example)
    
    filename = f"./iteration_final/basic_word_tokenized_500_test/tfrecords/{dataset_type}/{str(iter_num).zfill(4)}.tfrecord"
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_features_dataset)

In [4]:
def tf_serialize_example(f0, f1, f2, f3):
    tf_string = tf.py_function(serialize_example, (f0, f1, f2, f3), tf.string)
    return tf.reshape(tf_string, ())

In [5]:
def serialize_example(paper_title, journal, doc_type, targets):
    paper_title_list = tf.train.Int64List(value=paper_title.numpy().tolist())
    journal_list = tf.train.Int64List(value=journal.numpy().tolist())
    doc_type_list = tf.train.Int64List(value=doc_type.numpy().tolist())
    targets_list = tf.train.Int64List(value=targets.numpy().tolist())
    
    paper_title_feature = tf.train.Feature(int64_list = paper_title_list)
    journal_feature = tf.train.Feature(int64_list = journal_list)
    doc_type_feature = tf.train.Feature(int64_list = doc_type_list)
    targets_feature = tf.train.Feature(int64_list = targets_list)
    
    features_for_example = {
        'paper_title': paper_title_feature,
        'journal': journal_feature,
        'doc_type': doc_type_feature,
        'targets': targets_feature
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=features_for_example))
    
    return example_proto.SerializeToString()

In [6]:
def turn_part_file_into_tfrecord(base_path, dataset_type='train'):
    file_list = [x for x in os.listdir(f"{base_path}{dataset_type}") if x.endswith('parquet')]
    file_list.sort()
    print(f"There are {len(file_list)} files for {dataset_type}")
    for i, file_name in enumerate(file_list):
        data = pd.read_parquet(f"{base_path}{dataset_type}/{file_name}")
        print(f"_____File number: {i} ({data.shape[0]} samples)")
        create_tfrecords_dataset(data, i, dataset_type)

In [7]:
base_file_path = f"./iteration_final/basic_word_tokenized_500_test/tokenized_data/"

#### Without padding

In [8]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'train')

There are 50 files for train
_____File number: 0 (2109134 samples)


2021-11-13 10:04:46.279043: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-13 10:04:46.281935: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-13 10:05:56.631832: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


_____File number: 1 (2385744 samples)
_____File number: 2 (2137645 samples)
_____File number: 3 (2137100 samples)
_____File number: 4 (2417211 samples)
_____File number: 5 (2132194 samples)
_____File number: 6 (2595984 samples)
_____File number: 7 (2133842 samples)
_____File number: 8 (2135744 samples)
_____File number: 9 (2311716 samples)
_____File number: 10 (2131537 samples)
_____File number: 11 (2412375 samples)
_____File number: 12 (1826565 samples)
_____File number: 13 (1674089 samples)
_____File number: 14 (1673903 samples)
_____File number: 15 (1672946 samples)
_____File number: 16 (1673859 samples)
_____File number: 17 (1674658 samples)
_____File number: 18 (1675982 samples)
_____File number: 19 (2410204 samples)
_____File number: 20 (2591167 samples)
_____File number: 21 (1675654 samples)
_____File number: 22 (1673776 samples)
_____File number: 23 (2407402 samples)
_____File number: 24 (2409526 samples)
_____File number: 25 (1674963 samples)
_____File number: 26 (2587482 samp

In [9]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'val')

There are 10 files for val
_____File number: 0 (61298 samples)
_____File number: 1 (62022 samples)
_____File number: 2 (30502 samples)
_____File number: 3 (31457 samples)
_____File number: 4 (26913 samples)
_____File number: 5 (29725 samples)
_____File number: 6 (60653 samples)
_____File number: 7 (30778 samples)
_____File number: 8 (55072 samples)
_____File number: 9 (59489 samples)
CPU times: user 2min 35s, sys: 11 s, total: 2min 46s
Wall time: 2min 20s


In [10]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'test')

There are 5 files for test
_____File number: 0 (20395 samples)
_____File number: 1 (24338 samples)
_____File number: 2 (24756 samples)
_____File number: 3 (17072 samples)
_____File number: 4 (10677 samples)
CPU times: user 34.3 s, sys: 2.5 s, total: 36.8 s
Wall time: 30.8 s
